# Learned Sensing Matrix

We got an encoder-decoder like system set up using a similar architecture with DeepCodec. Now let's grab the sensing matrix, load it into MATLAB, and see if we can get better results using a learned sensing matrix on MNIST!

In [5]:
import torch
from torch import nn
from glob import glob

In [4]:
class WFUMNISTCodec(nn.Module):
    """A NN that definitely won't work, lmao, Test our dataloader."""
    def __init__(self, original_res, sensing_res):
        """Makes a FCN type architecture, takes in picture dimension d x d"""
        super(WFUMNISTCodec, self).__init__()
        
        self.m = sensing_res
        # Sensing matrix
        self.sensing = torch.nn.Linear(original_res * original_res, sensing_res * sensing_res, bias=False)
        
        self.conv1 = torch.nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 8, kernel_size=4, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(8, 4, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(4, 1, kernel_size=3, stride=1, padding=1)
        self.upsample = torch.nn.UpsamplingBilinear2d(size=(original_res, original_res))
        
    def forward(self, x):
        # Apply the sensing matrix without activation, reshape back to image-like for FCN
        x = self.sensing(x.view(x.shape[0], -1)).view(x.shape[0], 1, self.m, self.m)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.conv4(x)
        return self.upsample(x)

In [8]:
torchmodels = list(sorted(glob("/Users/wfu/Documents/MATLAB/AM254/models/wfunetsmall*.torchmodel")))

In [13]:
undersamplings = [int(t.replace(".torchmodel", "").split('_')[-1]) for t in torchmodels]

## Getting the Sensing Matrix

The sensing matrix is just the first fully connected layer that maps MNIST into a lower dimensional space. Let's return this and serialize it into something that MATLAB can load.

In [16]:
def get_sensing_from_model(model):
    params = list(model.parameters())
    return params[0].detach().cpu().numpy()

In [17]:
sensing_layers = [get_sensing_from_model(torch.load(t)) for t in torchmodels] 

/usr/local/lib/python3.6/site-packages/torch/serialization.py:367: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


AttributeError: module 'torch._C' has no attribute '_cuda_getDevice'

In [21]:
from scipy.io import savemat

In [23]:
savemat('sensing_mnist_100_784.mat', {'A': a})